In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84125825/84125825 [==============================] - 9s 0us/step


['test', 'imdb.vocab', 'train', 'imdbEr.txt', 'README']

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['unsupBow.feat',
 'urls_neg.txt',
 'labeledBow.feat',
 'urls_pos.txt',
 'unsup',
 'pos',
 'urls_unsup.txt',
 'neg']

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [4]:
batch_size = 1
seed = 12345
train_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
                            'aclImdb/train', batch_size=batch_size, 
                            validation_split=0.2,
                            subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [5]:
for text_batch, label_batch in train_ds:
    print(label_batch[0].numpy())
    print(text_batch[0].numpy().decode())
    break

1
When we talk Hollywood Hotel we could be talking about one of three things, the actual hotel, the radio program, and this film which was partially inspired by the first two. Dick Powell was the host of the Hollywood Hotel program on CBS radio network in which Louella Parsons dished out the weekly scoop on the stars.<br /><br />Powell and Parsons debuted the Hollywood Hotel program in 1934 so by 1937 it had its fair share of the radio audience. Powell hosted, sang, and kibitzed with Louella and her movie star guests. With the power she had with her column, she was able to get the various stars to go on and plug their latest films for nothing.<br /><br />Then the American Federation of Radio Artists stepped in and demanded she pay wages accordingly and they won the case. That ended the Hollywood Hotel program in 1938. Of course both Powell and Louella went on to other radio venues. The whole story is covered in the Tony Thomas book, The Films Of Dick Powell.<br /><br />But before the p

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.4 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

def convert_text_to_feature(review, tokenizer, max_length):  
    return tokenizer.encode_plus(review,
                                add_special_tokens=True,
                                max_length = max_length,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True)

def map_feature_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {"input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_masks,}, label

def encode_text(ds, tokenizer, max_length):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
        
    for review, label in ds:
        bert_input = convert_text_to_feature(review[0].numpy().decode(), tokenizer, max_length)
    
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label[0].numpy()])

    return tf.data.Dataset.from_tensor_slices(
                (input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_feature_to_dict)

In [ ]:

max_length = 512
learning_rate = 2e-5
epochs = 2

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
tokenizer.encode_plus(text_batch[0].numpy().decode(),
                      add_special_tokens = True,
                      max_length = max_length,
                      padding='max_length',
                      truncation=True,
                      return_attention_mask = True)

{'input_ids': [101, 2043, 2057, 2831, 5365, 3309, 2057, 2071, 2022, 3331, 2055, 2028, 1997, 2093, 2477, 1010, 1996, 5025, 3309, 1010, 1996, 2557, 2565, 1010, 1998, 2023, 2143, 2029, 2001, 6822, 4427, 2011, 1996, 2034, 2048, 1012, 5980, 8997, 2001, 1996, 3677, 1997, 1996, 5365, 3309, 2565, 2006, 6568, 2557, 2897, 1999, 2029, 10223, 8411, 13505, 9841, 2098, 2041, 1996, 4882, 23348, 2006, 1996, 3340, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 8997, 1998, 13505, 6006, 1996, 5365, 3309, 2565, 1999, 4579, 2061, 2011, 4347, 2009, 2018, 2049, 4189, 3745, 1997, 1996, 2557, 4378, 1012, 8997, 4354, 1010, 6369, 1010, 1998, 11382, 16313, 5422, 2007, 10223, 8411, 1998, 2014, 3185, 2732, 6368, 1012, 2007, 1996, 2373, 2016, 2018, 2007, 2014, 5930, 1010, 2016, 2001, 2583, 2000, 2131, 1996, 2536, 3340, 2000, 2175, 2006, 1998, 13354, 2037, 6745, 3152, 2005, 2498, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2059, 1996, 2137, 4657, 1997, 2557, 3324, 3706, 1999, 1998, 6303, 2016, 3477, 

In [ ]:
batch_size = 8

train_ds = encode_text(train_ds, tokenizer, max_length).shuffle(32).batch(batch_size)
val_ds = encode_text(val_ds, tokenizer, max_length).batch(batch_size)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[2].get_weights()

[array([[-0.01607964,  0.00781504],
        [ 0.00072904,  0.03182775],
        [-0.00905255,  0.02398794],
        ...,
        [ 0.02556369, -0.00047625],
        [ 0.00893582, -0.01035128],
        [-0.00827898,  0.01289442]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
)

history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/2
2500/2500 [==============================] - 2694s 1s/step - loss: 0.2436 - accuracy: 0.9013 - val_loss: 0.2122 - val_accuracy: 0.9210
Epoch 2/2
2500/2500 [==============================] - 2643s 1s/step - loss: 0.1250 - accuracy: 0.9568 - val_loss: 0.2226 - val_accuracy: 0.9148
